In [2]:
import warnings
warnings.filterwarnings("ignore")
# import missingno as mso
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
train_data = pd.read_csv('../input/heartbeat-signal/train.csv')
test_data = pd.read_csv('../input/heartbeat-signal/testA.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../input/heartbeat-signal/train.csv'

In [ ]:
train_data.head().append(train_data.tail())

In [ ]:
train_data.shape

In [ ]:
test_data.head().append(test_data.tail())

In [ ]:
test_data.shape

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
test_data.info()

In [ ]:
test_data.describe()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data['label'].value_counts()

In [ ]:
import scipy.stats as st
y = train_data['label']
plt.figure(1); plt.title("Default")
sns.distplot(y, rug=True, bins=20)
plt.figure(2); plt.title("Normal")
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title("Log Normal")
sns.distplot(y, kde=False, fit=st.lognorm)

In [ ]:
sns.distplot(train_data['label']);
print("Skewness: %f" % train_data['label'].skew())
print("Kurtosis: %f" % train_data['label'].kurt())

In [ ]:
train_data.skew(), train_data.kurt()

In [ ]:
sns.distplot(train_data.kurt(),color='orange',axlabel ='Kurtness')

In [ ]:
plt.hist(train_data['label'], orientation = 'vertical',histtype = 'bar', color ='red')

In [ ]:
import pandas_profiling
pfr = pandas_profiling.ProfileReport(train_data)
pfr.to_file("./example.html")

**TSFRESH**

In [ ]:
import tsfresh as tsf
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.head()

### TRAINING

In [ ]:
x = train_data.iloc[1:4]

In [ ]:
x["heartbeat_signals"].str.split(",", expand=True)

In [ ]:
x["heartbeat_signals"].str.split(",", expand=True).stack()

In [ ]:

time_x = x["heartbeat_signals"].str.split(",", expand=True).stack().reset_index()

time_x

In [ ]:
time_x = time_x.set_index('level_0')
time_x

In [ ]:
time_x.index.name = None
time_x

In [ ]:
time_x.rename(columns={"level_1":"time", 0:"heartbeat_signals"}, inplace=True)
time_x["heartbeat_signals"] = time_x["heartbeat_signals"].astype(float)
time_x

In [ ]:
train_heartbeat_df = train_data["heartbeat_signals"].str.split(",", expand=True).stack()
train_heartbeat_df = train_heartbeat_df.reset_index()
train_heartbeat_df = train_heartbeat_df.set_index("level_0")
train_heartbeat_df.index.name = None
train_heartbeat_df.rename(columns={"level_1":"time", 0:"heartbeat_signals"}, inplace=True)
train_heartbeat_df["heartbeat_signals"] = train_heartbeat_df["heartbeat_signals"].astype(float)

train_heartbeat_df

In [ ]:
train_data_label = train_data["label"]
train_data = train_data.drop("label", axis=1)
train_data = train_data.drop("heartbeat_signals", axis=1)
train_data

In [ ]:
train_data = train_data.join(train_heartbeat_df)

In [ ]:
train_data

In [ ]:
train_data[train_data['id']==1]


Time Series Feature extraction based on scalable hypothesis tests (https://tsfresh.readthedocs.io/en/latest/text/introduction.html)

In [ ]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters

settings = MinimalFCParameters()
train_features = extract_features(train_data, column_id='id', column_sort='time', default_fc_parameters=settings)
train_features

In [ ]:
from tsfresh.utilities.dataframe_functions import impute
impute(train_features)

In [ ]:
from tsfresh import select_features
train_features_filtered = select_features(train_features, train_data_label)
train_features_filtered

In [ ]:
# from tsfresh import extract_relevant_features

# train_featuresd_filtered = extract_relevant_features(train_data, train_data_label, column_id='id', column_sort='time'，default_fc_parameters=settings)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

import os
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

reduce memory usage

In [ ]:
def reduce_mem_usage(df):
    # DataFrame.memory_usage()
    start_mem = df.memory_usage().sum() / 1024**2 
    print(f"Memory usage of original dataframe is {start_mem:.2f} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min() 
            c_max = df[col].max()
           
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else: 
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
        
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization is: {end_mem:.2f}")
    reduce = 100 * (start_mem - end_mem) / start_mem
    print(f"Decreased by {reduce:.1f}%")
        
    return df

In [ ]:
data = pd.read_csv('../input/heartbeat-signal/train.csv')
data_list = []
for items in data.values: 
    data_list.append([items[0]] + [float(i) for i in items[1].split(',')] + [items[2]])  

data = pd.DataFrame(np.array(data_list)) 
data.columns = ['id'] + ['s_'+str(i) for i in range(len(data_list[0])-2)] + ['label']

data = reduce_mem_usage(data)

Memory usage of original dataframe is 157.93 MB
Memory usage after optimization is: 39.67
Decreased by 74.9%


In [ ]:
from sklearn.model_selection import cross_val_score
# 调objective
best_obj = dict()
for obj in objective:
    model = LGBMRegressor(objective=obj)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
    best_obj[obj] = score

# num_leaves
best_leaves = dict()
for leaves in num_leaves:
    model = LGBMRegressor(objective=min(best_obj.items(), key=lambda x:x[1])[0], num_leaves=leaves)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
    best_leaves[leaves] = score

# max_depth
best_depth = dict()
for depth in max_depth:
    model = LGBMRegressor(objective=min(best_obj.items(), key=lambda x:x[1])[0],
                          num_leaves=min(best_leaves.items(), key=lambda x:x[1])[0],
                          max_depth=depth)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()
    best_depth[depth] = score